# Initial Set-Up

In [6]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

# Infogroup
This is the infogroup code. It has most 

In [ ]:
file = "../../data/companyData/infogroupPublic.csv"
hq = pd.read_csv(file, encoding = 'unicode_escape')[['ticker','archive_version_year',
                                                    'company','city','state','zipcode',
                                                    'latitude','longitude','business_status_code']]
# hq = hq[hq.business_status_code == 1.0]

In [ ]:
hq.head()

In [ ]:
hq.columns

The code commented out below lets us scale if we do the full infogroup file.

In [3]:
import dask.dataframe as dd
df = dd.read_csv( "../../data/companyData/infogroup2010s.csv", assume_missing=True, 
                 dtype={'parent_number': 'object','parent_employee_size_code': 'object',
                       'parent_sales_volume_code': 'object',
                       'abi': 'object'})

In [4]:
df.head()

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)


,archive_version_year,abi,ticker,parent_number,company,address_line_1,city,state,zipcode,location_employee_size_code,location_sales_volume_code,primary_naics_code,sic_code,business_status_code,parent_employee_size_code,parent_sales_volume_code,cbsa_code,latitude,longitude
0,2010.0,000002907,NaN,004556080,QUINCY COMPRESSOR,701 N DOBSON AVE,BAY MINETTE,AL,36507.0,F,NaN,33391202.0,NaN,3.0,D,NaN,19300.0,30.88473,-87.79219
1,2010.0,000003178,NaN,389361817,TRONOX INC,1 KERR MCGEE RD,SAVANNAH,GA,31404.0,H,I,32599805.0,509901.0,2.0,H,NaN,42340.0,32.07764,-81.02519
2,2010.0,000004135,NaN,007530124,PPG INDUSTRIES INC,10800 S 13TH ST,OAK CREEK,WI,53154.0,H,I,32551011.0,509905.0,2.0,I,NaN,33340.0,42.84868,-87.92959
3,2010.0,000004358,NaN,441306552,GREENWALD INDUSTRIES,212 MIDDLESEX AVE,CHESTER,CT,6412.0,F,G,33399920.0,399903.0,2.0,B,NaN,25540.0,41.42210,-72.44318
4,2010.0,000004432,NaN,NaN,GUIDA-SEIBERT DAIRY CO,433 PARK ST,NEW BRITAIN,CT,6051.0,G,H,11212001.0,202398.0,9.0,NaN,NaN,25540.0,41.66500,-72.76954


In [ ]:
hq = hq[(hq.business_status_code == 1.0)]

In [ ]:
hq.shape

In [ ]:
hq.head()

In [ ]:
hq.to_csv('../../data/companyData/hqPublicAll.csv')

In [ ]:
hq.head()

# Create df

Convert into geospatial dataframe.

In [ ]:
geoHQ = gpd.GeoDataFrame(
    hq,
    geometry=gpd.points_from_xy(
        hq["longitude"],
        hq["latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

In [ ]:
geoHQ[geoHQ.archive_version_year == 1997].plot()

This is the most basic version of the HQ information. BS takes it a step further by filtering for firms where the HQ are responsible for at least a certain percentage of the overall sales.

In [7]:
os.getcwd()

'/Users/brianreed/Documents/supplyChain/extremes/extremesCleaningCode'

In [8]:
file = "../../data/companyData/infogroup.csv"
hqSales = pd.read_csv(file, encoding = 'unicode_escape')
hqSales = hqSales[hqSales.business_status_code == 1.0]

# hqSales = hq

FileNotFoundError: [Errno 2] No such file or directory: '../../data/companyData/infogroup.csv'

Overstate the size of a location relative to the parent: put parent at low end and location at high end of ranges.

In [ ]:
hqSales.loc[hqSales.parent_employee_size_code == 'A', 'parent_employee_size_code'] = 1
hqSales.loc[hqSales.parent_employee_size_code == 'B', 'parent_employee_size_code'] = 5
hqSales.loc[hqSales.parent_employee_size_code == 'C', 'parent_employee_size_code'] = 10
hqSales.loc[hqSales.parent_employee_size_code == 'D', 'parent_employee_size_code'] = 20
hqSales.loc[hqSales.parent_employee_size_code == 'E', 'parent_employee_size_code'] = 50
hqSales.loc[hqSales.parent_employee_size_code == 'F', 'parent_employee_size_code'] = 100
hqSales.loc[hqSales.parent_employee_size_code == 'G', 'parent_employee_size_code'] = 250
hqSales.loc[hqSales.parent_employee_size_code == 'H', 'parent_employee_size_code'] = 500
hqSales.loc[hqSales.parent_employee_size_code == 'I', 'parent_employee_size_code'] = 1000
hqSales.loc[hqSales.parent_employee_size_code == 'J', 'parent_employee_size_code'] = 5000
hqSales.loc[hqSales.parent_employee_size_code == 'K', 'parent_employee_size_code'] = 10000


hqSales.loc[hqSales.location_employee_size_code == 'A', 'location_employee_size_code'] = 4
hqSales.loc[hqSales.location_employee_size_code == 'B', 'location_employee_size_code'] = 9
hqSales.loc[hqSales.location_employee_size_code == 'C', 'location_employee_size_code'] = 19
hqSales.loc[hqSales.location_employee_size_code == 'D', 'location_employee_size_code'] = 49
hqSales.loc[hqSales.location_employee_size_code == 'E', 'location_employee_size_code'] = 99
hqSales.loc[hqSales.location_employee_size_code == 'F', 'location_employee_size_code'] = 249
hqSales.loc[hqSales.location_employee_size_code == 'G', 'location_employee_size_code'] = 499
hqSales.loc[hqSales.location_employee_size_code == 'H', 'location_employee_size_code'] = 999
hqSales.loc[hqSales.location_employee_size_code == 'I', 'location_employee_size_code'] = 4999
hqSales.loc[hqSales.location_employee_size_code == 'J', 'location_employee_size_code'] = 9999
hqSales.loc[hqSales.location_employee_size_code == 'K', 'location_employee_size_code'] = 10000


hqSales['location_employee_size_code'] = pd.to_numeric(hqSales['location_employee_size_code'], errors='coerce')
hqSales['parent_employee_size_code'] = pd.to_numeric(hqSales['parent_employee_size_code'], errors='coerce')
hqSales['employeesAtLocation'] = hqSales['location_employee_size_code']/hqSales['parent_employee_size_code']




In [ ]:
hqSales['employeesAtLocation'].describe()

In [ ]:
sum(hqSales.employeesAtLocation > 0.75)/hqSales.shape[0]

Filter down the location information to firms where we have at least 75% of employees at HQ.

In [ ]:
geoHQ = geoHQ[hqSales.employeesAtLocation > 0.75].reset_index()

In [ ]:
geoHQ.drop(columns = ['index','geometry']).to_csv('data/geoHQ.csv')

In [ ]:
geoHQ = pd.read_csv("../../data/geoHQ.csv")

In [ ]:
geoHQ.shape

In [ ]:
geoHQ.head()

In [ ]:
compustat1 = pd.read_csv("../../data/companyData/compustatChanges_All.csv").drop(columns = ['Unnamed: 0'])

compustat2 = pd.read_csv("../../data/companyData/compustatChanges_2010s.csv").drop(columns = ['Unnamed: 0','naics'])

compustat = compustat1.append(compustat2)

compustat.rename(columns = {'tic': 'TICKER'}, inplace = True)

print(compustat1.columns,compustat2.columns)

In [ ]:
print(compustat.shape, compustat.head())

In [ ]:
cstatCos = compustat[['year','companyName','TICKER']].drop_duplicates()
cstatCos = cstatCos[cstatCos.year > 1996]

geoHQ.rename(columns = {'archive_version_year': 'year'},inplace = True)

In [ ]:
geoHQ.head()

In [ ]:
allIG = cstatCos.merge(geoHQ)
allIG.to_csv("../../data/igHQ.csv")

# Weather Data
## *this is how we process the cmip6Data


In [ ]:
import netCDF4 as nc
import xarray as xr

In [ ]:
file = '../../../../../../../Volumes/backup2/dissData/cmip6Data/hist/pr/pr_day_MIROC6_historical_r7i1p1f1_gn_20100101-20141231.nc'
data = xr.open_dataset(file)   #  nc.Dataset(file)

In [ ]:
data

In [ ]:
geoHQ = gpd.GeoDataFrame(
    allIG,
    geometry=gpd.points_from_xy(
        allIG["longitude"],
        allIG["latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

In [ ]:
geoHQ['longitude']

In [ ]:
one_point = data["pr"].sel(lat=50, lon=50)

In [ ]:
pd.DataFrame(test.temperature[test.year == 2010]).plot()

Now go through and make this iterable for the entire list.

In [ ]:
i = 0
data.pr.sel(allIG.iloc[0,:], method='nearest')

In [ ]:
hqPost2010 = geoHQ[geoHQ.archive_version_year > 2010].reset_index()

In [ ]:
start = time.time()



tempsList = list()
for i in range(0,hqPost2010.shape[0]):
    nearbyTemps = temps.sel(hqPost2010.loc[i,['latitude','longitude']], method='nearest')
    
    tempsList.append(list(np.array(nearbyTemps.temperature[nearbyTemps.year == hqPost2010.archive_version_year[i]])))

    if (i%100 == 0):
        print(i)
    
print(time.time() - start)






In [ ]:
outfile =  'data/hqDailyTemperatures.pkl'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(tempsList, pickle_file)